In [25]:
#from thllib import flylib as flb
from matplotlib import pyplot as plt
import numpy as np
import scipy
#from thllib import util
import figurefirst as fifi
import scipy.signal
import local_project_functions as lpf
from IPython.display import SVG,display
import networkx as nx


import flylib as flb
#from thllib import flylib as flb
import flylib as flb
from matplotlib import pyplot as plt
import numpy as np
import scipy
from flylib import util
import figurefirst as fifi
import scipy.signal
import local_project_functions as lpf
from IPython.display import SVG,display
import pandas as pd
import networkx as nx

from scipy.stats import norm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

In [2]:
flynumbers = list(range(1389,1402))
fly_nums = range(1389,1392)
multi_fly_df = util.construct_multi_fly_df(fly_nums)



Making dataframe for fly number 1389
Making dataframe for fly number 1390
Making dataframe for fly number 1391


In [17]:
### to extract motifs from closed loop data
#import signal

In [3]:
multi_fly_df.head()

,t,stimulus,amp_diff,left_amp,right_amp,fly_num,wbf,experimental_condition,pr_left,tpd_left,...,hg2_right,hg3_right,hg1_right,i1_right,bkg_right,b1_right,b2_right,b3_right,iii24_right,hg4_right
0,0.000000,pretrial_stripe_fix,NaN,NaN,NaN,1389,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
1,0.020000,pretrial_stripe_fix,NaN,NaN,NaN,1389,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
2,0.040001,pretrial_stripe_fix,NaN,NaN,NaN,1389,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
3,0.060001,pretrial_stripe_fix,NaN,NaN,NaN,1389,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0
4,0.080002,pretrial_stripe_fix,NaN,NaN,NaN,1389,0.0,condition=pretrial_closed_loop;gain=-1,1.186526,1.537965,...,7.332649,0.0,2.973134,0.549872,22.93889,0.424018,0.284354,1.161884,0.0,0.0


In [ ]:
multi_fly_df

In [ ]:
def normd (muscle_data):
    x = muscle_data
    normalized = (x-min(x))/(max(x)-min(x))
    return normalized

In [ ]:
print(multi_fly_df.columns.values)
print(np.unique(multi_fly_df['stimulus']))

idx = (multi_fly_df['stimulus']=='cl_blocks, g_x=-1, g_y=0, b_x=-8, b_y=0, ch=True')& \
        ((multi_fly_df['amp_diff']>0.1)&(multi_fly_df['amp_diff']<0.104))


In [4]:
multi_fly_df['l_plus_r'] = multi_fly_df['left_amp'] + multi_fly_df['right_amp']
all_stim_df = multi_fly_df


In [5]:
def rad_to_deg (rad):
    r_deg = np.rad2deg(((rad)/10) *2*np.pi)
    return r_deg

In [6]:
def key_to_key(inkey):
    """map the input key from the cov_mtrx_triang_layout.svg into a set of keys 
    (a tuple of tuples) that can be used to construct signals from the fly data"""
    try:
        k1 = {'R':'right','L':'left'}[inkey[0]]
    except KeyError:
        if inkey[0] == 'w':
            return ('common','wb_freq')
        else:
            return
    k2 = inkey[1:]
    return (k1,k2)

In [21]:
#from scipy import medfilt

In [26]:
from scipy.signal import medfilt

In [29]:
def median_filter_all_columns(dataframe):
    df = pd.DataFrame()
    columns = dataframe.columns
    for column in columns:
        df[column]=scipy.signal.medfilt(dataframe.column, 3)
    return(df)

In [30]:
df = median_filter_all_columns(multi_fly_df)

AttributeError: 'DataFrame' object has no attribute 'column'

In [8]:
df = median_filter_all_columns(multi_fly_df)

In [ ]:
## normalize med filtered 

In [ ]:
#normalized = normed_multi_fly_df.loc[normed_multi_fly_df['stimulus']== 'yaw_left, g_x=12, g_y=0, b_x=0, b_y=0, ch=0']


In [ ]:

def normd(fly_num, muscle):
        df=multi_fly_df.loc[multi_fly_df['fly_num']==fly_num]
        a = len(df[muscle])
        m = df[muscle]
        #a_5p = a*5/100.
        #a_5pr = round(a_5p)
        #minimums_5p = arr.argsort()[:a_5pr]
        #maximums_5p = arr.argsort()[:-a_5pr]
        #minimums_5p = np.argpartition(df, a_5pr)
        print("1.5th percentile:   ", np.percentile(m, 1.5))
        muscle_min = np.percentile(m, 1.5)
        print("98.5th percentile:   ", np.percentile(m, 98.5))
        muscle_max = np.percentile(m, 98.5)

        normalized = (m-muscle_min)/(muscle_max-muscle_min)
        return normalized

In [ ]:
normed_multi_fly_df = pd.DataFrame()

for flynum in fly_nums:
    df=multi_fly_df.loc[multi_fly_df['fly_num']==flynum]
    for key in sorted_keys: 
        a = normd(df[key])
        df[key] = a
    normed_multi_fly_df = normed_multi_fly_df.append(df)